<a href="https://colab.research.google.com/github/teodoragrubor/projekatPPPO/blob/master/pokusaj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install segmentation_models_pytorch
! git clone https://github.com/Bjarten/early-stopping-pytorch.git
! mv ./early-stopping-pytorch ./lib

     |████████████████████████████████| 71kB 3.5MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 245kB 21.5MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp37-none-any.whl size=60966 sha256=9a57ae9c3737802024b23bdc39b96530fa7bf47c9f2a34f03bdc3a5032bb98ed
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-cp37-none-any.whl size=12420 sha256=ac409ba8c8b1695c3eb24bed3445ecefa1107d7a37e87e2716c7e0c9408b6f6a
  Stored in directory: /root/.cache/pip/wheels/42/1e/a9/2a578ba9ad04e776e80bf0f70d8a7f4c29ec0718b92d8f6ccd
Successfully built pretrainedmodels efficientnet-pytorch
Cloning into 'early-stopping-pytorch'...
remote: Enumerating objects: 92, done.
remote: Total 92 (delta 0), reused 0 (delta 0), pack-reused 92
Unpacking objects: 100% (92/92), done.


In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, sampler
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import torchvision
import cv2
import re
import segmentation_models_pytorch as smp
from lib.pytorchtools import *

In [ ]:
class NerveDataset(Dataset):
    def __init__(self, directory, pytorch=True, is_test=False):
        super().__init__()
        
        # Loop through the files in 'directory' folder and combine, into a dictionary, the masks
        self.files = []
        for file_name in directory.iterdir():
            
            if 'mask' in str(file_name):
                continue
                
            self.files.append(self.combine_files(file_name))
            
        # Sorting files list
        self.files = sorted(self.files, key=lambda file: int(re.search(r'\d+', str(file['image'])).group(0)))
        #print(self.files)
        
        self.pytorch = pytorch
        self.resize = torchvision.transforms.Resize((224,224),interpolation=Image.NEAREST)
        self.is_test = is_test
        
    def combine_files(self, file_name: Path):
        
        files = {
            'image': file_name, 
            'mask': '..' + str(file_name).split('.')[2] + '_mask.tif',
        }

        return files
                                       
    def __len__(self):
        
        return len(self.files)
     
    def open_as_array(self, idx, invert=False):
        
        raw_image = self.resize(Image.open(self.files[idx]['image']))
        raw_image = raw_image = np.stack([ np.array(raw_image) ], axis=2)
    
        if invert:
            raw_image = raw_image.transpose((2,0,1))
    
        # normalize
        return (raw_image / np.iinfo(raw_image.dtype).max)
    

    def open_mask(self, idx, add_dims=False):
        
        raw_mask = self.resize(Image.open(self.files[idx]['mask']))
        raw_mask = np.array(raw_mask)
        raw_mask = np.where(raw_mask==255, 1, 0)
        
        return np.expand_dims(raw_mask, 0) if add_dims else raw_mask
    
    def __getitem__(self, idx):
        
        x = torch.tensor(self.open_as_array(idx, invert=self.pytorch), dtype=torch.float32)
        
        if not self.is_test:    
            y = torch.tensor(self.open_mask(idx, add_dims=True), dtype=torch.torch.float32)
            return x, y
        
        return x
    
    def open_as_pil(self, idx):
        
        arr = 256*self.open_as_array(idx)
        
        return Image.fromarray(arr.astype(np.uint8), 'L')
    
    def __repr__(self):
        s = 'Dataset class with {} files'.format(self.__len__())

        return s

In [ ]:
images_path = Path('/content/drive/MyDrive/#8/train')
data = NerveDataset(images_path)
len(data)

IndexError: ignored